In [1]:
import yaml

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

N = config["rnn"]["simulation"]["N"]
timesteps = config["rnn"]["simulation"]["timesteps"]
dt = config["rnn"]["simulation"]["dt"]
alpha = config["rnn"]["simulation"]["alpha"]
beta = config["rnn"]["simulation"]["beta"]

seed = config["rnn"]["model"]["seed"]
train_size = config["rnn"]["model"]["train_size"]
val_size = config["rnn"]["model"]["val_size"]
test_size = config["rnn"]["model"]["test_size"]
window_size = config["rnn"]["model"]["window_size"]

loss = config["rnn"]["training"]["loss"]
save = config["rnn"]["data"]["save"]
load = config["rnn"]["data"]["load"]

hyperparameters = config["gridsearch"]["hyperparameters"]

In [3]:
import numpy as np
from functions import Simulation as SIM
from functions import MachineLearning as ML
import os

# Specify the folder path
folder_path = r"D:/School/Magistrska/data"

q_i = np.sin(np.linspace(0, np.pi*2, N))
p_i = np.zeros(N)

sim = SIM(q_i, p_i, N, timesteps, dt)

# Check if the folder is empty
if len(os.listdir(folder_path)) == 0:
    print("No saved data, running simulation.")

    qs, ps = sim.integrate(alpha, beta)
    
else:
    print("Found some saved data, skipping simulation.")
    
    qs = np.load(folder_path+"\data_qs_1.npy")
    ps = np.load(folder_path+"\data_ps_1.npy")
    
qs = qs[:, 1:-1]
ps = ps[:, 1:-1]

In [4]:
from sklearn.model_selection import train_test_split

X, y = ML.make_sequences(preprocess(qs), window_size)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_size/(1-val_size), shuffle=False)

In [5]:
import tensorflow as tf
tf.random.set_seed(seed)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, GRU

def build_model(learning_rate, hidden_units_per_layer, dropout):
    
    initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=seed)
    
    model = Sequential()
    model.add(InputLayer((window_size, N)))

    model.add(GRU(hidden_units_per_layer, activation="tanh", kernel_initializer=initializer))
    
    if dropout != 0:
        model.add(Dropout(dropout))
        
    model.add(Dense(N))
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)

    return model

In [6]:
import numpy as np
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.base import clone

# Define the custom accuracy score function
def custom_accuracy_score(y_true, y_pred):
    test_pred = np.copy(y_true[0])
    for k in range(len(y_true)):
        pred = model.predict(test_pred[np.newaxis, -window_size:], verbose=0)
        test_pred = np.concatenate((test_pred, pred), axis=0)
    test_pred = test_pred[window_size:]
    mse = mean_squared_error(postprocess(y_true), postprocess(test_pred))
    return -mse  # Minimize MSE, so negative sign is added

# Make a scorer object using the custom accuracy score function
custom_scorer = make_scorer(custom_accuracy_score)

# Create the model
model = KerasRegressor(build_fn=build_model, epochs=10, batch_size=32, verbose=0)

# Use the custom scorer in GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=hyperparameters, scoring=custom_scorer, cv=3)
grid_result = grid.fit(X_train, y_train)  # Replace X_train and y_train with your data

# Print the best results
print("Best score: {:.4f}".format(-grid_result.best_score_))
print("Best parameters:", grid_result.best_params_)

c:\Users\Grand\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Grand\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\Grand\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\Users\Grand\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "<ipython-input-6-66941032c11b>", line 10, in custom_accuracy_score
    pred = model.predict(test_pred[np.newaxis, -window_size:], verbose=0)
  

Best score: nan
Best parameters: {'batch_size': 18, 'dropout': 0, 'epochs': 10, 'hidden_units_per_layer': 8, 'learning_rate': 0.01}
